In [1]:
import pandas as pd
import numpy as np
import glob as glob
import os
import time
import pathlib

In [2]:
start_time = time.time()
uri_list = "cash-and-cash-equivalents"
url_symbol_name = "apple"
url_symbol = "aapl"
url_fin_metric = "cash-and-cash-equivalents"


In [3]:
def magnitude_num(number, currency_symbol):
    if len(str(number)) > 9:
        magnitude = number/1000000000
        magnitude_str = "{}{}{}".format(currency_symbol,round(magnitude,1),"B")
    elif len(str(number)) > 6:
        magnitude = number/1000000
        magnitude_str = "{}{}{}".format(currency_symbol,round(magnitude,1),"M")
    elif len(str(number)) > 3:
        magnitude = number/1000
        magnitude_str = "{}{}{}".format(currency_symbol,round(magnitude,1),"K")
    else:
        magnitude = number
        magnitude_str = "{}{}{}".format(currency_symbol,round(magnitude,1),"K")
    return magnitude_str

In [4]:
company_profiles = pd.read_csv("../reference_data/Company_Profiles.csv")#, encoding='cp1252')
fin_statements_matching = pd.read_csv("../reference_data/Financial_Statements_Reference_Matching.csv")#, encoding='cp1252')
currency_symbol = list(company_profiles[company_profiles['symbol']=="{}".format(url_symbol.upper())]['currency symbol'])[0]
company_profiles_col = ['symbol',
                        'long name',
                        'currency',
                        'exchange',
                        'industry',
                        'description',
                        'sector',
                        'country',
                        'ipo date',
                        'short name']
company_profiles = company_profiles[company_profiles_col]  

In [5]:
profiles_dict = {}
profiles_value = company_profiles[company_profiles['symbol']=="{}".format(url_symbol.upper())].values.tolist()[0]

titles_bs = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Balance Sheet"]['Title'])
titles_cf = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Cash Flow Statement"]['Title'])
titles_is = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Income Statement"]['Title'])

urls_bs = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Balance Sheet"]['URL'])
urls_cf = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Cash Flow Statement"]['URL'])
urls_is = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Income Statement"]['URL'])

In [6]:
for n, profiles_col in enumerate(company_profiles_col):
    key = profiles_col
    value = profiles_value[n]
    profiles_dict[key] = value 

if "{}".format(url_fin_metric) in urls_is:
    fin_statement_dir = "Income Statement"
elif "{}".format(url_fin_metric) in urls_bs:
    fin_metric_pos = urls_bs.index("{}".format(url_fin_metric))
    fin_statement_dir = "Balance Sheet"
    fin_statement_cols = titles_bs
    cols = titles_bs
    fin_metric_title = fin_statement_cols[fin_metric_pos]

elif "{}".format(url_fin_metric) in urls_cf:
    fin_statement_dir = "Cash Flow Statement"

else:
    pass


In [7]:
# start_time = time.time()
csv_file = glob.glob("../Charts_TenDollarData/financial_statements/data/Historical Financial Statements/*/year/{}/*~{}~*".format(fin_statement_dir, url_symbol.upper()))[-1]

df = pd.read_csv(csv_file) #.format("NLOK"))[-1]
# df = df[df['date'].notna()]#fillna(method='ffill')
df = df[0:].iloc[::-1]#.dropna()
df_fin_statement = df
#region Pandas data manipulation
matching_row = fin_statements_matching[fin_statements_matching['URL']=="{}".format(url_fin_metric)]
fin_metric_title = list(matching_row['Title'])[0]
fin_metric_name = list(matching_row['Name'])[0]
print("fin metric name", fin_metric_name)
sorted_metric = df["{}".format(fin_metric_name)].sort_values()

fin metric name cashAndCashEquivalents


In [8]:



# sorted_metric(int(len(sorted_metric)*(n/100)))
quarters = round(len(sorted_metric)/4)
bottom_25 = sorted_metric[len(sorted_metric)-1-quarters]
top_25 = sorted_metric[quarters-1]
max_num = sorted_metric.max()
min_num = sorted_metric.min()
mean = sorted_metric.mean()
std_dev = sorted_metric.std()
std_dev_str = "+/-{}{}%".format(currency_symbol,round(abs((std_dev-mean)/mean)*100,1))
mean_str = magnitude_num(mean, currency_symbol)
max_str = magnitude_num(max_num, currency_symbol)
min_str = magnitude_num(min_num, currency_symbol)
bottom_25_str = magnitude_num(bottom_25, currency_symbol)
top_25_str = magnitude_num(top_25, currency_symbol)

lifetime_sum_all_metric = df["{}".format(fin_metric_name)].sum()
lifetime_sum_all_metric = magnitude_num(lifetime_sum_all_metric,currency_symbol)

latest_num = df["{}".format(fin_metric_name)][0]
first_num = list(df["{}".format(fin_metric_name)])[-1]
pct_chg = (latest_num - first_num)/first_num
historical_pct_chg = str(round(pct_chg*100, 1))

In [9]:
if pct_chg>=0:
    pct_chg_str = "+{}%".format(pct_chg)
elif pct_chg<0:
    pct_chg_str = "-{}%".format(pct_chg)
else:
    pct_chg_str = ""
# historical_pct_chg = str(round(pct_chg*100, 1))
max_min_pct_diff = ((max_num-min_num)/min_num)
if max_min_pct_diff>=0:
    max_min_pct_diff_str = "+{}%".format(round(max_min_pct_diff)*100,1)
elif max_min_pct_diff<0:
    max_min_pct_diff_str = "-{}%".format(round(max_min_pct_diff)*100,1)
else:
    max_min_pct_diff_str = ""

In [10]:
df_fin_statement['Quarter & Year'] =(df_fin_statement['date'].astype(str).str[0:4]).astype(int)
latest_year = list((df_fin_statement['date'].astype(str).str[0:4]))[0]
earliest_year = list((df_fin_statement['date'].astype(str).str[0:4]))[-1]
earliest_metric = (list(df["{}".format("{}".format(fin_metric_name))])[-1])
latest_metric = (list(df["{}".format("{}".format(fin_metric_name))])[0])


In [11]:
titles_list = ['Date','Symbol','Filing Date','Accepted Date','Period','SEC Filing Link'][:]

for x in reversed(titles_bs):
    if x in titles_list:
        titles_bs.remove(x)
        print(x, "is in")
    else:
        pass
#         print(x, "is not in")        

SEC Filing Link is in
Period is in
Accepted Date is in
Filing Date is in
Symbol is in
Date is in


In [12]:
titles_bs.append('Quarter & Year') 
df_fin_statement = df_fin_statement.drop([ 'Unnamed: 0','date','symbol','fillingDate','acceptedDate','period','link'],axis=1)
df_fin_statement.columns = titles_bs


In [13]:

df_fin_statement = df_fin_statement[cols]

df_fin_statement.index = df_fin_statement['Quarter & Year']

million = 1000000
billion = 1000000000
fin_metric_history = df_fin_statement['{}'.format(fin_metric_title)]
if list(fin_metric_history)[0]  > billion:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/billion).round(decimals=2)

elif list(fin_metric_history)[0]  > million:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/million).round(decimals=2)

    pass
else:
    pass

In [19]:
vars_names_list = []
vars_values_list = []
# for name in vars().keys():
#   vars_names_list.append(name)
for name, value in globals().items():
    vars_names_list.append(name)
    vars_values_list.append(value)
vars_names_list    

['__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__builtin__',
 '__builtins__',
 '_ih',
 '_oh',
 '_dh',
 'In',
 'Out',
 'get_ipython',
 'exit',
 'quit',
 '_',
 '__',
 '___',
 'pyforest',
 'json',
 'getsizeof',
 'NamespaceMagics',
 '_nms',
 '_Jupyter',
 'np',
 '_getsizeof',
 '_getshapeof',
 'var_dic_list',
 '_i',
 '_ii',
 '_iii',
 '_i1',
 'pd',
 'glob',
 'os',
 'time',
 'pathlib',
 '_i2',
 'start_time',
 'uri_list',
 'url_symbol_name',
 'url_symbol',
 'url_fin_metric',
 '_i3',
 'magnitude_num',
 '_i4',
 'company_profiles',
 'fin_statements_matching',
 'currency_symbol',
 'company_profiles_col',
 '_i5',
 'profiles_dict',
 'profiles_value',
 'titles_bs',
 'titles_cf',
 'titles_is',
 'urls_bs',
 'urls_cf',
 'urls_is',
 '_i6',
 'n',
 'profiles_col',
 'key',
 'value',
 'fin_metric_pos',
 'fin_statement_dir',
 'fin_statement_cols',
 'cols',
 'fin_metric_title',
 '_i7',
 'csv_file',
 'df',
 'df_fin_statement',
 'matching_row',
 'fin_metric_name',
 'sorted_metric',
 '_i8',

In [21]:
"asdasd"[0:100]

'asdasd'

In [25]:
vars_values_list_100 = [str(x)[0:50] for x in vars_values_list]

In [30]:
vars_df = pd.DataFrame({'vars_names_list':vars_names_list,'vars_values_list_100':vars_values_list_100})
# vars_values_list_100
vars_df.to_csv("../reference_data/flask_variables.csv")

In [18]:

df_pct_chg = df_fin_statement
pct_chg_cols = (df_fin_statement.select_dtypes(include=['number']).pct_change(-1))
df_pct_chg_str = df_pct_chg.drop(list(pct_chg_cols), axis=1)
df_pct_chg = df_pct_chg_str.join(pct_chg_cols)[list(df_fin_statement)]

pd.set_option('display.float_format', '{:.2f}'.format)
df_pct_chg_t = df_pct_chg.transpose()
df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
df_pct_chg_t = df_pct_chg_t.iloc[1:]

df_pct_chg_t = df_pct_chg.transpose()
df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
df_pct_chg_t = df_pct_chg_t.iloc[1:]
df_pct_chg_t['']=df_pct_chg_t.index
df_pct_chg_t.index = range(len(df_pct_chg_t))

cols = list(df_pct_chg_t.columns)
cols = [cols[-1]] + cols[:-1]
df_pct_chg_t = df_pct_chg_t[cols]


df_t = df_fin_statement.transpose()
df_t.columns = list(df_fin_statement['Quarter & Year'])
df_t = df_t.iloc[1:]
df_t['']=df_t.index
df_t.index = range(len(df_t))

cols = list(df_t.columns)
cols = [cols[-1]] + cols[:-1]
df_t = df_t[cols]
#endregion

df_pct = df_pct_chg_t.to_html().replace('<table','<table class="df_tableBoot" id="df_myTable1"')# dt-responsive" id="df_myTable"')

df_t = df_t[df_t.columns[::-1]]
cols = list(df_t.columns)
cols = [cols[-1]] + cols[:-1]
df_t = df_t[cols]


def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False
df_n = df_fin_statement[df_fin_statement.applymap(isnumber)]
df_n[df_n < 2] = np.nan
df_n_sum = pd.DataFrame(df_n.sum())
df_n_sum[df_n_sum == 0] = ""
new_header = df_n_sum.iloc[0] #grab the first row for the header
df_n_sum = df_n_sum[1:] #take the data less the header row
df_n_sum.columns = new_header #set the header row as the df header
df_n_sum.index = range(len(df_n_sum))
df_t = pd.merge(df_n_sum, df_t, left_index=True, right_index=True,suffixes=('Total: {} - {}'.format(latest_year,earliest_year), 'Line Items'))
df_t = df_t[0:25]



col_list = []
n=0
while n<len(list(df_t))-0:
    if n<6:
            col_item = '<col id="col_item_{}" class="col_item_class first_7_col">'.format(n)
    else:
        col_item = '<col id="col_item_{}" class="col_item_class">'.format(n)
    col_list.append(col_item)
    n+=1
col_list_str = ''.join(map(str, col_list))
df_html = df_t.to_html().replace('border="1" class="dataframe">','class="df_tableBoot" id="df_myTable" border="1" class="dataframe"><colgroup>{}</colgroup>'.format(col_list_str))
# df_table_html = df_fin_statement[['{}'.format(fin_metric_title)]].iloc[::-1].transpose().to_html()#.replace("\n","")
df_tall = df.iloc[::-1]
# df_tall.index = df['Quarter & Year']
# df_tall = df.index.shift(-1)
# df_tall = df_tall.reset_index()
df_html_tall = df_tall[['{}'.format('Quarter & Year'),'{}'.format(fin_metric_name)]].to_html(index=False)
df_html_tall = df_html_tall.replace('border="1" class="dataframe">','class="abc" id="df_myTable" border="1" class="dataframe">')#.replace("'","")
# df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("{}".format("["),"")
df_html_tall = df_html_tall #render_template_string(df_html_tall)
df_html = df_html.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
#  df_html_tall = df_html_tall.replace("\n","")
# df_html_tall.replace("\n",'">')
df_html_tall = df_html_tall.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall[0:]

df = df[['date',"{}".format(fin_metric_name)]].dropna() #.fillna(0)#.fillna(method='bfill')
df['date'] = pd.to_datetime(df['date']).values.astype(np.int64) // 10 ** 6
full_path = csv_file.split(' ~ ')
path = pathlib.PurePath(full_path[0])
print("Nothing took {} seconds".format(time.time() - start_time))
total_seconds = ((time.time() - start_time))

labels = [
    'JAN', 'FEB', 'MAR', 'APR',
    'MAY', 'JUN', 'JUL', 'AUG',
    'SEP', 'OCT', 'NOV', 'DEC'
]
labels = list(df['date'])#[0:19]

values = [
    967.67, 1190.89, 1079.75, 1349.19,
    2328.91, 2504.28, 2873.83, 4764.87,
    4349.29, 6458.30, 9907, 16297
]

values = list(df["{}".format(fin_metric_name)])#[0:19]

colors = [
    "#F7464A", "#46BFBD", "#FDB45C", "#FEDCBA",
    "#ABCDEF", "#DDDDDD", "#ABCABC", "#4169E1",
    "#C71585", "#FF4500", "#FEDCBA", "#46BFBD"]
df_table_html = df_tall[['{}'.format(fin_metric_name)]].iloc[::-1].transpose().to_html()#.replace("\n","")
present_num = magnitude_num(int(latest_metric),currency_symbol)
latest_num = "${}".format(df["{}".format(fin_metric_name)][0])


Nothing took 267.5588176250458 seconds


In [13]:
import requests
response = requests.get("http://www.google.com")
response = requests.get("http://127.0.0.1:5000/aapl-stock/apple/cash-and-cash-equivalents")
print(response.content)

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"\n  "http://www.w3.org/TR/html4/loose.dtd">\n<html>\n  <head>\n    <title>NameError: name \'start_time\' is not defined // Werkzeug Debugger</title>\n    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css"\n        type="text/css">\n    <!-- We need to make sure this has a favicon so that the debugger does\n         not by accident trigger a request to /favicon.ico which might\n         change the application state. -->\n    <link rel="shortcut icon"\n        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">\n    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=jquery.js"></script>\n    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>\n    <script type="text/javascript">\n      var TRACEBACK = 1834011132928,\n          CONSOLE_MODE = false,\n          EVALEX = true,\n          EVALEX_TRUSTED = false,\n          SECRET = "t3wb8w77FxDSu3AUMLC3";\n    </

In [14]:
import urllib.request

myurl = "http://127.0.0.1:5000/aapl-stock/apple/cash-and-cash-equivalents"

# # GET is the default action
# response = urllib.request.urlopen(myurl)  

# # Output from the GET assuming response code was 200
# data = response.read()  

with urllib.request.urlopen("http://127.0.0.1:5000/aapl-stock/apple/cash-and-cash-equivalents") as url:
    s = url.read()
    # I'm guessing this would output the html source code ?
    print(s)

HTTPError: HTTP Error 500: INTERNAL SERVER ERROR

In [36]:
import sys
print (sys.executable)
!{sys.executable} -m pip install requests_testadapter --upgrade

C:\Users\Tom\quantconnect\Scripts\python.exe
  Created wheel for requests-testadapter: filename=requests_testadapter-0.3.0-py3-none-any.whl size=4692 sha256=3b4e290717e164afb25bf11ba2539e3bd60e289e56538b60df70bcbe0399a55e
  Stored in directory: c:\users\tom\appdata\local\pip\cache\wheels\f6\85\0d\6b2d456f81dd6168b161711ebaca57d61fbe0744c52bcdb96b
Successfully built requests-testadapter


In [ ]:
from requests_testadapter import Resp

class LocalFileAdapter(requests.adapters.HTTPAdapter):
    def build_response_from_file(self, request):
        file_path = request.url[7:]
        with open(file_path, 'rb') as file:
            buff = bytearray(os.path.getsize(file_path))
            file.readinto(buff)
            resp = Resp(buff)
            r = self.build_response(request, resp)

            return r

    def send(self, request, stream=False, timeout=None,
             verify=True, cert=None, proxies=None):

        return self.build_response_from_file(request)

requests_session = requests.session()
requests_session.mount('file://', LocalFileAdapter())
requests_session.get('file://<some_local_path>')

In [15]:
import requests

os.environ['NO_PROXY'] = '127.0.0.1'
r = requests.get('http://127.0.0.1:5000/aapl=stock/apple/cash-and-cash-equivalents')
print(r.content)

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n'


In [40]:
import requests
os.environ['NO_PROXY'] = '127.0.1'

r=requests.get("http://127.0.01:5000/aapl=stock/apple/cash-and-cash-equivalents")

ConnectionError: HTTPConnectionPool(host='127.0.01', port=5000): Max retries exceeded with url: /aapl=stock/apple/cash-and-cash-equivalents (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000020480EA1580>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [16]:
# BASE_URL='http://127.0.0.1:8000'
BASE_URL = "http://127.0.0.1:5000/aapl=stock/apple/cash-and-cash-equivalents/"
BASE_URL = "http://127.0.0.1:5000/aapl=stock/apple/cash-and-cash-equivalents"
resp=requests.get(BASE_URL)
# print(resp.status_code)
# print(resp.json())

In [17]:
resp.text

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n'

In [19]:
req = requests.post("http://127.0.0.1:5000/aapl=stock/apple/cash-and-cash-equivalents", 'html.parser')
req

<Response [404]>

In [48]:
from pypac import PACSession
session = PACSession()
# session.get('https://127.0.0.1/aapl=stock/apple/cash-and-cash-equival
session.get("http://127.0.01:5000/aapl=stock/apple/cash-and-cash-equivalents")
# <Response [200]>

ConnectionError: HTTPConnectionPool(host='127.0.01', port=5000): Max retries exceeded with url: /aapl=stock/apple/cash-and-cash-equivalents (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000020480EFD550>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
from pypac import PACSession
session = PACSession()
session.get('https://127.0.0/aapl=stock/apple/cash-and-cash-equivalents')
# <Response [200]>

In [ ]:
from requests_testadapter import Resp

class LocalFileAdapter(requests.adapters.HTTPAdapter):
    def build_response_from_file(self, request):
        file_path = request.url[7:]
        with open(file_path, 'rb') as file:
            buff = bytearray(os.path.getsize(file_path))
            file.readinto(buff)
            resp = Resp(buff)
            r = self.build_response(request, resp)

            return r

    def send(self, request, stream=False, timeout=None,
             verify=True, cert=None, proxies=None):

        return self.build_response_from_file(request)

requests_session = requests.session()
requests_session.mount('file://', LocalFileAdapter())
requests_session.get('file://<some_local_path>')

In [12]:








df_fin_statement = df_fin_statement[cols]

df_fin_statement.index = df_fin_statement['Quarter & Year']

million = 1000000
billion = 1000000000
fin_metric_history = df_fin_statement['{}'.format(fin_metric_title)]
if list(fin_metric_history)[0]  > billion:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/billion).round(decimals=2)


elif list(fin_metric_history)[0]  > million:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/million).round(decimals=2)

    pass
else:
    pass

df_pct_chg = df_fin_statement
pct_chg_cols = (df_fin_statement.select_dtypes(include=['number']).pct_change(-1))
df_pct_chg_str = df_pct_chg.drop(list(pct_chg_cols), axis=1)
df_pct_chg = df_pct_chg_str.join(pct_chg_cols)[list(df_fin_statement)]

pd.set_option('display.float_format', '{:.2f}'.format)
df_pct_chg_t = df_pct_chg.transpose()
df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
df_pct_chg_t = df_pct_chg_t.iloc[1:]

df_pct_chg_t = df_pct_chg.transpose()
df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
df_pct_chg_t = df_pct_chg_t.iloc[1:]
df_pct_chg_t['']=df_pct_chg_t.index
df_pct_chg_t.index = range(len(df_pct_chg_t))

cols = list(df_pct_chg_t.columns)
cols = [cols[-1]] + cols[:-1]
df_pct_chg_t = df_pct_chg_t[cols]


df_t = df_fin_statement.transpose()
df_t.columns = list(df_fin_statement['Quarter & Year'])
df_t = df_t.iloc[1:]
df_t['']=df_t.index
df_t.index = range(len(df_t))

cols = list(df_t.columns)
cols = [cols[-1]] + cols[:-1]
df_t = df_t[cols]
#endregion

df_pct = df_pct_chg_t.to_html().replace('<table','<table class="df_tableBoot" id="df_myTable1"')# dt-responsive" id="df_myTable"')

df_t = df_t[df_t.columns[::-1]]
cols = list(df_t.columns)
cols = [cols[-1]] + cols[:-1]
df_t = df_t[cols]


def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False
df_n = df_fin_statement[df_fin_statement.applymap(isnumber)]
df_n[df_n < 2] = np.nan
df_n_sum = pd.DataFrame(df_n.sum())
df_n_sum[df_n_sum == 0] = ""
new_header = df_n_sum.iloc[0] #grab the first row for the header
df_n_sum = df_n_sum[1:] #take the data less the header row
df_n_sum.columns = new_header #set the header row as the df header
df_n_sum.index = range(len(df_n_sum))
df_t = pd.merge(df_n_sum, df_t, left_index=True, right_index=True,suffixes=('Total: {} - {}'.format(latest_year,earliest_year), 'Line Items'))
df_t = df_t[0:25]



col_list = []
n=0
while n<len(list(df_t))-0:
    if n<6:
            col_item = '<col id="col_item_{}" class="col_item_class first_7_col">'.format(n)
    else:
        col_item = '<col id="col_item_{}" class="col_item_class">'.format(n)
    col_list.append(col_item)
    n+=1
col_list_str = ''.join(map(str, col_list))
df_html = df_t.to_html().replace('border="1" class="dataframe">','class="df_tableBoot" id="df_myTable" border="1" class="dataframe"><colgroup>{}</colgroup>'.format(col_list_str))
# df_table_html = df_fin_statement[['{}'.format(fin_metric_title)]].iloc[::-1].transpose().to_html()#.replace("\n","")
df_tall = df.iloc[::-1]
# df_tall.index = df['Quarter & Year']
# df_tall = df.index.shift(-1)
# df_tall = df_tall.reset_index()
df_html_tall = df_tall[['{}'.format('Quarter & Year'),'{}'.format(fin_metric_name)]].to_html(index=False)
df_html_tall = df_html_tall.replace('border="1" class="dataframe">','class="abc" id="df_myTable" border="1" class="dataframe">')#.replace("'","")
# df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("{}".format("["),"")
df_html_tall = df_html_tall #render_template_string(df_html_tall)
df_html = df_html.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
#  df_html_tall = df_html_tall.replace("\n","")
# df_html_tall.replace("\n",'">')
df_html_tall = df_html_tall.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall[0:]

df = df[['date',"{}".format(fin_metric_name)]].dropna() #.fillna(0)#.fillna(method='bfill')
df['date'] = pd.to_datetime(df['date']).values.astype(np.int64) // 10 ** 6
full_path = csv_file.split(' ~ ')
path = pathlib.PurePath(full_path[0])
print("Nothing took {} seconds".format(time.time() - start_time))
total_seconds = ((time.time() - start_time))

labels = [
    'JAN', 'FEB', 'MAR', 'APR',
    'MAY', 'JUN', 'JUL', 'AUG',
    'SEP', 'OCT', 'NOV', 'DEC'
]
labels = list(df['date'])#[0:19]

values = [
    967.67, 1190.89, 1079.75, 1349.19,
    2328.91, 2504.28, 2873.83, 4764.87,
    4349.29, 6458.30, 9907, 16297
]

values = list(df["{}".format(fin_metric_name)])#[0:19]

colors = [
    "#F7464A", "#46BFBD", "#FDB45C", "#FEDCBA",
    "#ABCDEF", "#DDDDDD", "#ABCABC", "#4169E1",
    "#C71585", "#FF4500", "#FEDCBA", "#46BFBD"]
df_table_html = df_tall[['{}'.format(fin_metric_name)]].iloc[::-1].transpose().to_html()#.replace("\n","")
present_num = magnitude_num(int(latest_metric),currency_symbol)
latest_num = "${}".format(df["{}".format(fin_metric_name)][0])

return render_template('current_ratio.html', company_symbol = profiles_dict['symbol'],\
                        company_long_name = profiles_dict['long name'],\
                        company_currency = profiles_dict['currency'],\
                        company_exchange = profiles_dict['exchange'],\
                        company_industry = profiles_dict['industry'],\
                        company_description = profiles_dict['description'],\
                        company_sector = profiles_dict['sector'],\
                        company_country = profiles_dict['country'],\
                        company_ipo_date = profiles_dict['ipo date'],\
                        company_short_name = profiles_dict['short name'],\
                        first_num = first_num,\
                        latest_num = latest_num,\
                        historical_pct_chg = historical_pct_chg,\
                        lifetime_sum_all_metric = lifetime_sum_all_metric,\
                        mean_str = mean_str,\
                        max_str = max_str,\
                        min_str = min_str,\
                        std_dev_str = std_dev_str,\
                        bottom_25_str = bottom_25_str,\
                        top_25_str = top_25_str,\
                        earliest_year = earliest_year,\
                        latest_year = latest_year,\
                        earliest_metric = earliest_metric,\
                        latest_metric = latest_metric,\
                        max_min_pct_diff_str = max_min_pct_diff_str, df_bs_table_html = [df_table_html],df_html_tall = [df_html_tall],fin_metric_name = fin_metric_title,\
                        df_date = df['date'].to_list(), df_rev = df["{}".format(fin_metric_name)].to_list(),\
                        df_json  =df.to_numpy().tolist(),\
                        table_pct = [df_pct],\
                        tables=[df_html],\
                        titles=df.columns.values,\
                        total_time=total_seconds,\
                        place_name=url_symbol,\
                        max=17000,\
                        labels=labels,\
                        values=values)


ValueError: Length mismatch: Expected axis has 41 elements, new values have 42 elements